In [22]:
import pandas as pd
import numpy as np


In [23]:
df = pd.read_csv("data/berlin_crime_final.csv")

In [24]:
df.shape

(21879, 10)

In [25]:
df.dtypes

LOR_ID           float64
Bezirk            object
Bezirksregion     object
CrimeType         object
Fallzahl         float64
year               int64
HZ               float64
geometry          object
lon              float64
lat              float64
dtype: object

In [26]:
df.head()

,LOR_ID,Bezirk,Bezirksregion,CrimeType,Fallzahl,year,HZ,geometry,lon,lat
0,11001.0,Mitte,Tiergarten Süd,Straftaten insgesamt,5475.0,2015,38575.0,MULTIPOLYGON (((13.373297493618068 52.50373208...,13.358181,52.510452
1,11002.0,Mitte,Regierungsviertel,Straftaten insgesamt,10313.0,2015,102068.0,MULTIPOLYGON (((13.389973675173831 52.50757271...,13.390625,52.514159
2,11003.0,Mitte,Alexanderplatz,Straftaten insgesamt,23524.0,2015,45938.0,MULTIPOLYGON (((13.423027234898033 52.51212031...,13.402246,52.520850
3,11004.0,Mitte,Brunnenstraße Süd,Straftaten insgesamt,4583.0,2015,16175.0,MULTIPOLYGON (((13.387830945640253 52.53339054...,13.393361,52.533828
4,12005.0,Mitte,Moabit West,Straftaten insgesamt,7428.0,2015,16612.0,MULTIPOLYGON (((13.343436370833128 52.52601880...,13.332072,52.530752


# Know the Data

In [27]:
df.Bezirk.value_counts()

Bezirk
Treptow-Köpenick              3060
Pankow                        2448
Tempelhof-Schöneberg          2295
Lichtenberg                   1989
Neukölln                      1836
Reinickendorf                 1836
Charlottenburg-Wilmersdorf    1683
Mitte                         1530
Spandau                       1377
Marzahn-Hellersdorf           1377
Friedrichshain-Kreuzberg      1224
Steglitz-Zehlendorf           1224
Name: count, dtype: int64

In [28]:
df.Bezirksregion.value_counts()

Bezirksregion
Heerstraße                    306
Grünau                        153
Oberschöneweide               153
Niederschöneweide             153
Adlershof                     153
                             ... 
Halensee                      153
Schmargendorf                 153
Grunewald                     153
Otto-Suhr-Allee/Kantstraße    153
MV Süd                        153
Name: count, Length: 142, dtype: int64

In [29]:
df.CrimeType.value_counts()

CrimeType
Straftaten insgesamt                                      1287
Fahrrad diebstahl                                         1287
Rauschgift delikte                                        1287
Sach beschädigung durch Graffiti                          1287
Sach beschädigung insgesamt                               1287
Brand stiftung                                            1287
Branddelikte insgesamt                                    1287
Wohnraum einbruch                                         1287
Diebstahl an/aus Kfz                                      1287
Raub                                                      1287
Diebstahl von Kraftwagen                                  1287
Diebstahl insgesamt                                       1287
Freiheits beraubung, Nötigung, Bedrohung, Nachstellung    1287
Gefährl. und schwere Körper verletzung                    1287
Körper verletzungen insgesamt                             1287
Straßenraub, Handtaschen raub                

In [5]:
df.isna().sum().sort_values(ascending=False)


HZ               2431
LOR_ID              0
Bezirk              0
Bezirksregion       0
CrimeType           0
Fallzahl            0
year                0
geometry            0
lon                 0
lat                 0
dtype: int64

In [7]:
df.duplicated().sum()


np.int64(0)

In [10]:
df["year"].nunique()


10

In [16]:
df["Bezirksregion"].nunique()


142

In [ ]:
df.groupby("year")["Bezirksregion"].nunique()


year
2015    142
2016    142
2017    142
2018    142
2019    142
2020    142
2021    142
2022    142
2023    142
2024    142
Name: Bezirksregion, dtype: int64